In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
from causalinference import CausalModel

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams['figure.figsize'] = 10, 8

In [2]:
# read data from the file
df = pd.read_csv('malocclusion.csv')  

In [3]:
df.head(5)

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0


## Estimate the effect of Treatment on Growth

<b>Task 1: Select variables to condition on</b>

In the causal relationships graph all undirected paths between Treatment and Growth are block by colliders except:
1.	Treatment -> Unobserved Confounders -> dT -> Growth
2.	Treatment -> Unobserved Confounders -> Growth

Ideally, we would condition on Unobserved Confounders and this would block both paths but we don't have this data.
<b>I will condition on dT</b>. This will block path 1 and we will have path 2 as unblocked backdoor.

<b>Task 2: ATE and ATET calulation</b>

I will use propensity score weighting to calculate ATE and propensity score matching to calculate ATET

ATE:  0.2083 with 95% CI (0.051, 0.365)

ATET: 0.300 with 95% CI (0.046, 0.553)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

# classifier to estimate the propensity score
cls = LogisticRegression()

# calibration of the classifier
cls = CalibratedClassifierCV(cls)

X = df[['dT']]
y = df['Treatment']
cls.fit(X, y)
df['e1'] = cls.predict_proba(X)[:,1].tolist()
df.head()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment,e1
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0,0.486891
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0,0.335406
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0,0.486891
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0,0.751201
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0,0.486891


In [5]:
df['w1'] = df['Treatment'] / df['e1'] + (1 - df['Treatment']) / (1 - df['e1'])

In [6]:
m = smf.wls('Growth ~ Treatment + dT', data=df, weights=df['w1'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     5.881
Date:                Sat, 30 Apr 2022   Prob (F-statistic):            0.00353
Time:                        15:36:29   Log-Likelihood:                -99.753
No. Observations:                 143   AIC:                             205.5
Df Residuals:                     140   BIC:                             214.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4515      0.092      4.930      0.0

In [7]:
causal = CausalModel(
    Y=df['Growth'].values, # outcome
    D=df['Treatment'].values, # treatment
    X=df['e1'].values
)
causal.est_via_matching(bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.320      0.119      2.678      0.007      0.086      0.554
           ATC      0.337      0.155      2.174      0.030      0.033      0.640
           ATT      0.300      0.130      2.314      0.021      0.046      0.553



C:\SM\Anaconda3\envs\HSE\lib\site-packages\causalinference\estimators\matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


## Estimate the effect of Treatment on dANB

<b>Task 1: Select variables to condition on</b>

In the causal relationships graph all paths between Treatment and dANB are block by colliders except:
1.	Treatment -> Unobserved Confounders -> dT -> Growth -> dANB
2.	Treatment -> Unobserved Confounders -> Growth -> dANB

<b>I will condition on Growth</b>. This will block both paths.

<b>Task 2: ATE and ATET calulation</b>

I will use propensity score weighting to calculate ATE and propensity score matching to calculate ATET

ATE:  1.8570 with 95% CI (1.380, 2.334)

ATET: 1.852 with 95% CI (1.382, 2.3223)

In [8]:
# classifier to estimate the propensity score
cls = LogisticRegression()

# calibration of the classifier
cls = CalibratedClassifierCV(cls)

X = df[['Growth']]
y = df['Treatment']
cls.fit(X, y)
df['e2'] = cls.predict_proba(X)[:,1].tolist()
df.head()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment,e1,w1,e2
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0,0.486891,1.948904,0.34318
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0,0.335406,1.504679,0.42124
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0,0.486891,1.948904,0.34318
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0,0.751201,4.019305,0.34318
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0,0.486891,1.948904,0.34318


In [9]:
df['w2'] = df['Treatment'] / df['e2'] + (1 - df['Treatment']) / (1 - df['e2'])

In [10]:
m = smf.wls('dANB ~ Treatment + Growth', data=df, weights=df['w2'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.386
Model:                            WLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     44.06
Date:                Sat, 30 Apr 2022   Prob (F-statistic):           1.44e-15
Time:                        15:37:20   Log-Likelihood:                -253.68
No. Observations:                 143   AIC:                             513.4
Df Residuals:                     140   BIC:                             522.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5572      0.205     -7.609      0.0

In [11]:
causal = CausalModel(
    Y=df['dANB'].values, # outcome
    D=df['Treatment'].values, # treatment
    X=df['e2'].values
)
causal.est_via_matching(bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.856      0.237      7.829      0.000      1.392      2.321
           ATC      1.860      0.240      7.761      0.000      1.390      2.330
           ATT      1.852      0.240      7.723      0.000      1.382      2.322



C:\SM\Anaconda3\envs\HSE\lib\site-packages\causalinference\estimators\matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef
